In [1]:
import os, re, glob, requests, gzip, urllib.request, time, zipfile, shutil, html, unidecode
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm_notebook as tqdm
from bs4 import BeautifulSoup

In [2]:
os.chdir('..')

---

read files

In [3]:
files = [f for f in os.listdir('/Users/maltemax/Desktop/files_sec') if f.endswith('zip')]

In [4]:
# with open(os.path.join(unzip_location, file, 'tag.tsv'), 'r') as f:
#     lis = [line.split('\t') for line in f]
# d = pd.DataFrame(lis)
# tag = pd.DataFrame(lis)
# tag.columns = tag.loc[0].tolist()
# tag = tag.drop(0)
# tag.replace(r'\n',' ', regex=True)

In [5]:
files_location = '/Users/maltemax/Desktop/files_sec'
unzip_location = '/Users/maltemax/Desktop/temp_unzip'
if not os.path.exists(unzip_location):
    os.makedirs(unzip_location)

collect = []

for file in tqdm(files):
    with zipfile.ZipFile(os.path.join(files_location, file), 'r') as zip_file:
        zip_file.extractall(os.path.join(unzip_location, file))
    
    # sub has all submissions for the given period
    sub = pd.read_csv(os.path.join(unzip_location, file, 'sub.tsv'), sep = '\t')
    sub = sub.loc[sub['form'] == '10-K']
    sub = sub[['adsh', 'cik', 'name', 'sic', 'period', 'fy', 'filed', 'instance']]
    
    # num has numerical values, one row for each tagged item
    num = pd.read_csv(os.path.join(unzip_location, file, 'num.tsv'), sep = '\t')
    num = num.loc[num['tag'].notna()]
    num['tag'] = num['tag'].str.lower()
    num = num.loc[num['tag'].str.contains('valuationallowance', flags = re.IGNORECASE)]
    
    # tag has tag information (official description, custom, etc.)
    tag = pd.read_csv(os.path.join(unzip_location, file, 'tag.tsv'), sep = '\t', on_bad_lines = 'warn')
    tag = tag.loc[tag['tag'].notna()]
    tag['tag'] = tag['tag'].str.lower()
    tag = tag.drop_duplicates('tag')
    tag = tag[['tag', 'custom', 'tlabel', 'doc']]
    
    # taking sub as the basis so that firms with no tags are retained
    # add num and tag information
    sub = sub.merge(right = num, how = 'left', on = 'adsh')
    sub = sub.merge(right = tag, how = 'left', on = 'tag')
    
    collect.append(sub)
    
    # delete the unzipped directory
    shutil.rmtree(os.path.join(unzip_location, file))

  0%|          | 0/78 [00:00<?, ?it/s]

/var/folders/z4/0rcsgf6524x_7ns1b5_sllfm0000gn/T/ipykernel_67656/2429272046.py:18: DtypeWarning: Columns (9,12) have mixed types. Specify dtype option on import or set low_memory=False.
  num = pd.read_csv(os.path.join(unzip_location, file, 'num.tsv'), sep = '\t')
/var/folders/z4/0rcsgf6524x_7ns1b5_sllfm0000gn/T/ipykernel_67656/2429272046.py:18: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  num = pd.read_csv(os.path.join(unzip_location, file, 'num.tsv'), sep = '\t')
/var/folders/z4/0rcsgf6524x_7ns1b5_sllfm0000gn/T/ipykernel_67656/2429272046.py:18: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  num = pd.read_csv(os.path.join(unzip_location, file, 'num.tsv'), sep = '\t')
/var/folders/z4/0rcsgf6524x_7ns1b5_sllfm0000gn/T/ipykernel_67656/2429272046.py:18: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  num = pd.read_csv(os.pat

/var/folders/z4/0rcsgf6524x_7ns1b5_sllfm0000gn/T/ipykernel_67656/2429272046.py:18: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  num = pd.read_csv(os.path.join(unzip_location, file, 'num.tsv'), sep = '\t')
/var/folders/z4/0rcsgf6524x_7ns1b5_sllfm0000gn/T/ipykernel_67656/2429272046.py:13: DtypeWarning: Columns (38) have mixed types. Specify dtype option on import or set low_memory=False.
  sub = pd.read_csv(os.path.join(unzip_location, file, 'sub.tsv'), sep = '\t')
/var/folders/z4/0rcsgf6524x_7ns1b5_sllfm0000gn/T/ipykernel_67656/2429272046.py:18: DtypeWarning: Columns (9,12) have mixed types. Specify dtype option on import or set low_memory=False.
  num = pd.read_csv(os.path.join(unzip_location, file, 'num.tsv'), sep = '\t')
/var/folders/z4/0rcsgf6524x_7ns1b5_sllfm0000gn/T/ipykernel_67656/2429272046.py:18: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  num = pd.read_csv(os.pa

In [6]:
df = pd.concat(collect, ignore_index = True)

/var/folders/z4/0rcsgf6524x_7ns1b5_sllfm0000gn/T/ipykernel_67656/3824935784.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(collect, ignore_index = True)


In [7]:
df['url_instance'] = ('https://www.sec.gov/Archives/edgar/data/' +
                      df['cik'].astype(str) + '/' +
                      df['adsh'].str.replace('-', '') + '/' +
                      df['instance'])

In [8]:
df.to_csv(os.path.join(os.getcwd(), '3_pipeline', '1_intermediate', 'va_data_raw.tsv'),
          sep = '\t', index = False)

In [9]:
# save meta file to disk
meta = df.drop_duplicates('adsh')[['adsh', 'cik', 'name', 'sic', 'period', 'fy',
                                 'filed', 'instance', 'url_instance']]
# calculate fiscal
meta['period'] = pd.to_datetime(meta['period'], format = '%Y%m%d')
meta['fyear'] = np.where(meta['period'].dt.month <= 5,
                         meta['period'].dt.year - 1,
                         meta['period'].dt.year)
meta.to_csv(os.path.join(os.getcwd(), '3_pipeline', '1_intermediate', 'va_data_info.tsv'),
            sep = '\t', index = False)

---

separate df into:

1. obs for which the standard tag VA or a different VA tag is available
2. obs which do not feature any tag: do they really have no valuation allowance?

In [126]:
df = pd.read_csv(os.path.join(os.getcwd(), '3_pipeline', '1_intermediate', 'va_data_raw.tsv'),
                 sep = '\t',)

/var/folders/z4/0rcsgf6524x_7ns1b5_sllfm0000gn/T/ipykernel_67656/2897716670.py:1: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(os.getcwd(), '3_pipeline', '1_intermediate', 'va_data_raw.tsv'),


In [127]:
df['period'] = pd.to_datetime(df['period'], format = '%Y%m%d')

In [128]:
# calculate fiscal
df['fyear'] = np.where(df['period'].dt.month <= 5,
                       df['period'].dt.year - 1,
                       df['period'].dt.year)

In [133]:
df = df.loc[df['fyear'].isin(range(2009, 2024 + 1))]

In [134]:
df['adsh'].nunique()

88608

In [135]:
tags = ['deferredtaxassetsvaluationallowance',
        'deferredtaxassetsvaluationallowancenoncurrent',
 'deferredtaxassetsvaluationallowancecurrent',
 'valuationallowanceamount', 'valuationallowance'
 'lessvaluationallowance', 'deferredtaxliabilitiesvaluationallowance',
 'deferredtaxassetsdomesticvaluationallowance',
 'deferredtaxassetsforeignvaluationallowance', 'deferredtaxassetvaluationallowance',
 'deferredtaxassetsvaluationallowances', 'valuationallowance1',
 'deferredtaxvaluationallowance', 'deferredtaxassetsvaluationallowance1',
 'deferredtaxassetsvaluationallowanceforeign',
 'deferredtaxassetsvaluationallowancedomestic',
 'deferredtaxassetsvaluationallowancefederal',
 'deferredtaxassetsvaluationallowancecurrentforeign',
 'deferredtaxassetsvaluationallowancecurrentfederal']

In [136]:
has_va = df.loc[df['tag'].isin(tags), 'adsh'].unique().tolist()

In [137]:
len(has_va)

60207

In [138]:
not_found = df.loc[df['tag'].isna()]

In [139]:
len(not_found)

20866

---

cleaning of: 2. obs for which no tag with string "valuation allowance" is in XBRL

approach: for these obs, check all tables in tax footnote: only if no table contains the string "valuation allowance", assign a zero

an example of such a company is here: the 10-K HTML shows that they have a VA (https://www.sec.gov/Archives/edgar/data/1462566/000126246314000593/biolog2013k.htm), but they did not tag it; I can use their XBRL file (https://www.sec.gov/Archives/edgar/data/1462566/000126246314000593/biol-20131231.xml), take out the income tax disclosure tag and check for all tables therein; and then delete if the string "valuation allowance" appears


*remaining problems:* there are some firms that do not include a detailed schedule of deferred tax assets but mention in the text that they have established a full valuation allowance for DTA. examples:
- https://www.sec.gov/Archives/edgar/data/1535079/000100201415000128/tpoi10k-12312014.htm
- https://www.sec.gov/ix?doc=/Archives/edgar/data/1852707/000157587224000990/bfyw-20240228.htm
- https://www.sec.gov/ix?doc=/Archives/edgar/data/1851612/000119312523087706/d624110d10k.htm

**how deal with these?**

check whether 'valuation allowance' appears in tables of tax footnotes:

In [140]:
external = '/Volumes/T7 Shield/research_data/us_xbrl_instances'
out = []

for i, r in tqdm(not_found.iterrows(), total = len(not_found)):
    path = os.path.join(external, f"{r['adsh']}.xml")
    with open(path, 'r') as text_in:
        text = text_in.read()
        # unescape HTML; otherwise opening HTML elements are encoded
        text = html.unescape(text)
        soup = BeautifulSoup(text, 'xml')
    
    # find tax footnote (in virtually all cases, this yields a list of 1 element)
    footnotes = soup.find_all(re.compile('^(?:us-gaap:)?IncomeTaxDisclosureTextBlock$', re.I))
    
    mentions_va = 0
    table_count = 0
    for footnote in footnotes:
        # identify all tables in footnote
        tables = footnote.find_all('table')
        for table in tables:
            table_count += 1
            table_text = unidecode.unidecode(table.text)
            table_text = table_text.lower()
            # following LM (Textual Analysis in Accounting and Finance: A Survey),
            # only include tables with at least 10% digits
            no_numbers = len(re.findall('\d', table_text))
            no_alphabetical = len(re.findall('[a-z]', table_text))
            
            # if table without alphabetical characters, continue with next table
            if no_alphabetical == 0:
                continue
            
            if (no_numbers / no_alphabetical) > 0.1:
                # if table contains the string 'valuation allowance',
                # set mentions_va to 1
                if re.search('valuation allowance', table_text):
                    mentions_va = 1
            else:
                pass
    
    out.append((r['adsh'], r['url_instance'], len(footnotes), table_count,  mentions_va))

  0%|          | 0/20866 [00:00<?, ?it/s]

In [141]:
check_zeros = pd.DataFrame(out, columns = ['adsh', 'url_instance', 'no_footnotes', 'no_tables', 'mentions_va'])

In [142]:
check_zeros.to_csv(os.path.join(os.getcwd(), '3_pipeline', '1_intermediate', 'va_data_check_zeros.tsv'),
                   sep = '\t', index = False)

read if already executed

In [143]:
check_zeros = pd.read_csv(os.path.join(os.getcwd(), '3_pipeline', '1_intermediate', 'va_data_check_zeros.tsv'),
                          sep = '\t')

delete observations from `not_found` for which `mentions_va == 1` in `check_zeros` (i.e., observations for which, in any table in the tax footnote, the string 'valuation allowance' does appear)

In [144]:
not_found = not_found.loc[~not_found['adsh'].isin(check_zeros.loc[check_zeros['mentions_va'] == 1, 'adsh'])]

cleaning of: 1. obs for which the standard tag is available

In [145]:
va = df.loc[df['tag'].isin(tags)].copy()

In [146]:
# checked: missing values should be zero VAs
va['value'] = va['value'].fillna(0)

In [147]:
# replace observations that have a negative VA amount by its absolute value
# CONFIRM THIS: I have checked a couple of these obs and saw that this is just because
# the companies report the VA in negative form
va['value'] = va['value'].abs()

In [148]:
va['ddate'] = pd.to_datetime(va['ddate'], format = '%Y%m%d')

In [149]:
# restrict to entries that are about the current fiscal year
# as companies also often report VA of last year
va = va.loc[(va['period'].dt.year == va['ddate'].dt.year) &
            (va['period'].dt.month == va['ddate'].dt.month)]

In [150]:
len(va)

83793

reports sometimes contain the same amount twice: once with highest precision in DTA table, once in the text (with lower precision); approach: round all VA amounts to lowest precision and then keep only the first of all duplicate observations

note: the SEC file contains unscaled values, so I'm not using the scale parameter, but values with different levels of precision have different leves of rounding to decimal points, which is what I use below

In [151]:
# replace INF (shown as 32767) by zero; INF means that it shows exact amounts, no rounding
# which is equal to dcml of 0
# some documents also appear to have -32768, which I also set to 0
va['dcml'] = np.where((va['dcml'] == 32767) | (va['dcml'] == -32768), 0, va['dcml'])

In [152]:
# there are only very few cases with a dcml of > 0 (what do they mean?)
va = va.loc[va['dcml'] <= 0]

In [153]:
va['min_decimal'] = va.groupby('adsh')['dcml'].transform('min')

In [154]:
# round all amounts to lowest level of precision used in report
va['va_min_accuracy'] = np.round(va['value'] / (10 ** abs(va['min_decimal'])), 0)

In [155]:
va = va.drop_duplicates(subset = ['cik', 'fyear', 'va_min_accuracy'], keep = 'first')


check if duplicates do not come from the fact that companies report disaggregated amounts as well as totals; approach: check whether values are sum of all other values, and in that case keep the largest value (which is likely the sum of all others)

In [156]:
def is_sum_of_numbers(number, numbers):
    if number == (sum(numbers) - number):
        return True
    else:
        return False

In [157]:
temp_dict = va.groupby('adsh')['va_min_accuracy'].unique().to_dict()

In [158]:
va_sum_of_others = {}

for k, v in temp_dict.items():
    va_sum_of_others[k] = max([is_sum_of_numbers(number = n, numbers = v) for n in v])

In [159]:
va_sum_of_others = pd.DataFrame(va_sum_of_others, index = [0]).T.reset_index()

In [160]:
va_sum_of_others.columns = ['adsh', 'va_sum_of_others']

In [161]:
va = va.merge(right = va_sum_of_others, how = 'left', on = 'adsh')

In [162]:
va['va_max'] = va.groupby('adsh')['value'].transform('max')

In [163]:
va['drop'] = np.where((va['va_sum_of_others'] == True) &
                      (va['value'] != va['va_max']), 1, 0)

In [164]:
va = va.loc[va['drop'] == 0]

In [165]:
va = va.drop(columns = ['min_decimal', 'va_min_accuracy', 'va_max', 'va_sum_of_others', 'drop'])

check whether duplicate comes from the fact that companies use non-standard tags to differentiate between jurisdictional origin of DTA (and hence, VA):

sometimes companies differentiate between the current and noncurrent portions of VA, or domestic vs foreign, or federal vs foreign

In [166]:
combos = [('deferredtaxassetsvaluationallowancenoncurrent',
           'deferredtaxassetsvaluationallowancecurrent'),
          ('deferredtaxassetsdomesticvaluationallowance',
           'deferredtaxassetsforeignvaluationallowance'),
          ('deferredtaxassetsvaluationallowancecurrentfederal',
           'deferredtaxassetsvaluationallowancecurrentforeign')]

In [167]:
for combo in combos:
    # identify reports that have the two tags AND ONLY the two tags
    # otherwise I would drop companies that, e.g., only use
    # 'deferredtaxassetsvaluationallowancenoncurrent' as their only tag to tag VAs
    va['both_present'] = np.where((va['tag'] == combo[0]) |
                                  (va['tag'] == combo[1]), 1, 0)
    va['both_present'] = va.groupby('adsh')['both_present'].transform('sum')
    va['total_tags'] = va.groupby('adsh')['tag'].transform('nunique')
    save = va.loc[(va['both_present'] == 2) & (va['total_tags'] == 2)]
    
    # from va, drop unnecessary columns and drop those for which the tags will be summed up
    va = va.drop(columns = ['both_present', 'total_tags'])
    va = va.loc[~va['adsh'].isin(save['adsh'])]
    
    # sum up values and merge back info from 'save'
    aggregated_values = save.groupby('adsh')['value'].sum().reset_index()
    aggregated_values = aggregated_values.merge(right = save.drop_duplicates('adsh').drop(columns = ['both_present', 'total_tags', 'value']),
                                                on = 'adsh', how = 'left')
    # overwrite value of tag to be able to idenfity individually-calculated values later
    aggregated_values['tag'] = 'CALCULATED_SUM'
    
    # concatenate va (from which the values that are summed up individually have been dropped above) and
    # the individually-summed up values
    va = pd.concat([va, aggregated_values])

it might be that a company has two VA tags left at this point: 1 that shows a zero (e.g., the current portion of the VA) and a non-zero value (e.g., the non-current portion of the VA). therefore: if there are only two tags left and one of the two is a zero, sum up the amounts manually

In [168]:
va['va_zero'] = np.where(va['value'] == 0, 1, 0)

In [169]:
va['va_zero_sum'] = va.groupby('adsh')['va_zero'].transform('sum')

In [170]:
va['total_tags'] = va.groupby('adsh')['tag'].transform('count')

In [171]:
aggregated_values = va.loc[(va['va_zero_sum'] == 1) & (va['total_tags'] == 2)]

In [172]:
aggregated_values = aggregated_values.groupby('adsh')['value'].sum().reset_index()

In [173]:
save = va.loc[va['adsh'].isin(aggregated_values['adsh'])]

In [174]:
va = va.loc[~va['adsh'].isin(aggregated_values['adsh'])]

In [175]:
aggregated_values = aggregated_values.merge(right = save.drop_duplicates('adsh').drop(columns = ['va_zero', 'va_zero_sum', 'total_tags', 'value']),
                                            on = 'adsh', how = 'left')
# overwrite value of tag to be able to idenfity individually-calculated values later
aggregated_values['tag'] = 'CALCULATED_SUM'

In [176]:
va = pd.concat([va, aggregated_values])

drop remaining duplicates at cik-fyear level (not keeping any)

In [177]:
va = va.drop_duplicates(subset = ['cik', 'fyear'], keep = False)

In [178]:
va['value'] = va['value'] / 1000000

In [179]:
len(va)

55648

add a valuation allowance of zero for all reports where the code does not find a VA

plus, add indicator for inferred zero (vs. a real zero) >> when code does not find VA, zero is inferred; otherwise real zero

In [180]:
va['zero_inferred'] = 0

In [181]:
not_found['tag'] = 'NO_VA_TAG_FOUND'

In [182]:
not_found['adsh'].nunique()

20037

In [183]:
va = pd.concat([va, not_found], ignore_index = True)

/var/folders/z4/0rcsgf6524x_7ns1b5_sllfm0000gn/T/ipykernel_67656/4105074280.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  va = pd.concat([va, not_found], ignore_index = True)


In [184]:
# add 1 if zero_inferred is missing (which it is only in the `not_found` dataframe)
va['zero_inferred'] = np.where(va['zero_inferred'].isna(), 1, va['zero_inferred'])

In [185]:
# only obs from `not_found` have a `value` that is missing; replace with zero
# these are observations that don't have the deferredtaxassetsvaluationallowance tag
# AND also no other tag that contains 'valuationallowance'
va['value'] = np.where(va['value'].isna(), 0, va['value'])

In [186]:
va['value'] = pd.to_numeric(va['value'])

In [187]:
va = va[['adsh', 'fyear', 'tag', 'value', 'zero_inferred']]

In [188]:
va = va.rename(columns = {'value' : 'va'})

In [189]:
len(va)

75685

In [190]:
va.to_csv(os.path.join(os.getcwd(), '3_pipeline', '1_intermediate', 'va_data_clean.tsv'),
          sep = '\t', index = False)